In [7]:
library(tidyverse)
library(broom)
library(repr)
library(digest)
library(infer)
library(gridExtra)
library(readxl)
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ dials        1.0.0     ✔ tune         1.0.0
✔ modeldata    1.0.1     ✔ workflows    1.0.0
✔ parsnip      1.0.1     ✔ workflowsets 1.0.0
✔ recipes      1.0.1     ✔ yardstick    1.1.0
✔ rsample      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ gridExtra::combine() masks dplyr::combine()
✖ scales::discard()    masks purrr::discard()
✖ dplyr::filter()      masks stats::filter()
✖ recipes::fixed()     masks stringr::fixed()
✖ dplyr::lag()         masks stats::lag()
✖ yardstick::spec()    masks readr::spec()
✖ recipes::step()      masks stats::step()
• Use tidymodels_prefer() to resolve common conflicts.



In [14]:
crime_data_2021 <- read.csv("https://github.com/Haobo11/stat201-group-project/raw/main/crimedata_csv_Hastings-Sunrise_2021.csv")
crime_data_2022 <- read.csv("https://github.com/Haobo11/stat201-group-project/raw/main/crimedata_csv_Hastings-Sunrise_2022.csv")

head(crime_data_2021)
head(crime_data_2022)

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,Theft from Vehicle,2021,8,17,21,57,1XX RENFREW ST,Hastings-Sunrise,496804.9,5459011
2,Theft from Vehicle,2021,9,20,20,45,20XX CASSIAR ST,Hastings-Sunrise,497740.0,5457015
3,Theft from Vehicle,2021,3,5,3,30,20XX CASSIAR ST,Hastings-Sunrise,497740.0,5457017
4,Theft from Vehicle,2021,2,15,12,0,21XX KAMLOOPS ST,Hastings-Sunrise,496024.4,5456995
5,Theft from Vehicle,2021,6,28,12,0,21XX LILLOOET ST,Hastings-Sunrise,497165.9,5456990
6,Theft from Vehicle,2021,4,14,3,0,21XX NANAIMO ST,Hastings-Sunrise,495887.6,5456922


,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,Theft from Vehicle,2022,5,3,22,0,1XX KOOTENAY ST,Hastings-Sunrise,498134.7,5458960
2,Theft from Vehicle,2022,5,26,7,30,1XX N RENFREW ST,Hastings-Sunrise,496800.4,5459316
3,Theft from Vehicle,2022,1,10,18,0,1XX NANAIMO ST,Hastings-Sunrise,495893.3,5459013
4,Theft from Vehicle,2022,9,25,17,55,1XX RENFREW ST,Hastings-Sunrise,496795.9,5458959
5,Theft from Vehicle,2022,4,29,20,0,20XX KAMLOOPS ST,Hastings-Sunrise,496024.4,5457091
6,Theft from Vehicle,2022,8,27,2,0,20XX NANAIMO ST,Hastings-Sunrise,495887.8,5457043
